In [4]:
import ROOT as r
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import math
import numpy as np
import colorsys
from scipy import optimize as sciopt
import os
import lmfit
import uncertainties as u
import dataset as ds
import datasetXtals as dsX
import fit_functions as funcs
import importlib

In [5]:
plt.style.use(hep.style.ATLAS)
plt.style.use({"axes.labelsize":'36', 'lines.linewidth':'2', 'lines.markersize':'2', 'xtick.labelsize':'16'})

In [39]:
energy_lim = [1000, 2400]
low_E = str(energy_lim[0] / 1000)
high_E = str(energy_lim[1] / 1000)

time_lim = [250, 450]
low_T = str(time_lim[0])
high_T = str(time_lim[1])

In [49]:
sixtyhour = ds.DataSet("1A", "60h", r.TFile("./data/60hXtalsPosition_v2.root"), energy_lim)
highkick  = ds.DataSet("1B", "HighKick", r.TFile("./data/highKickXtalsPosition_v2.root"), energy_lim)
nineday   = ds.DataSet("1C", "9day", r.TFile("./data/9dayXtalsPosition_v2.root"), energy_lim)
endgame   = ds.DataSet("1D", "EndGame", r.TFile("./data/endGameXtalsPosition_v2.root"), energy_lim)
run2C     = ds.DataSet("2C", "run2C", r.TFile("./data/run2CXtalsPosition_v2.root"), energy_lim)

datasets = {"1A": sixtyhour, "1B":highkick, "1C":nineday, "1D":endgame, "2C": run2C}
#datasets = {"1D":endgame}

for name, dataset in datasets.items():
    dataset.set_up_calos(step_length=1, verbose=False)

In [50]:
df = datasets['1A'].calos[1].data['randomized']

errors = df[df['TimeBin'] > 200]['MeanError']
n = len(errors)
error = np.sqrt((errors / n).pow(2).sum())
print(error)

0.033381749152577515


In [51]:

names = ['1A', '1B', '1C', '1D', '2C']

for name in names:

    this_data = pd.DataFrame(columns = ['mean', 'error', 'caloNum'])

    for num, calo in datasets[name].calos.items():
        
        df = calo.data['randomized']
        mean = df[df['TimeBin'] > time_lim[0]]['Mean'].mean()
        
        errors = df[df['TimeBin'] > time_lim[0]]['MeanError']
        n = len(errors)
        error = np.sqrt((errors / n).pow(2).sum())

        p = {'mean':    mean,
             'error':   error,
             'caloNum': num}
        
            
        this_data.loc[num] = p
        

    #print(this_data)
    this_data.to_csv("./mean_csv/" + name + ".csv")


In [52]:
rand_data = {}
names = ['1A', '1B', '1C', '1D', '2C']

for name in names:
    rand_data[name] = pd.read_csv("./mean_csv/" + name + ".csv")



fig, ax = plt.subplots(1,1)
fig.set_size_inches(15, 15)


rand_data_2 = {}
rand_data_2['1A'] = rand_data['1A']
rand_data_2['1B'] = rand_data['1B']
rand_data_2['1C'] = rand_data['1C']
rand_data_2['1D'] = rand_data['1D']
rand_data_2['2C'] = rand_data['2C']

for name, df in rand_data_2.items():
    ax.errorbar(df['caloNum'], df['mean'], yerr=df['error'], label=name)
    


ax.set_xlabel('calo number')
ax.set_ylabel('y-Mean [mm]')


#ax[0].set_ylim(28, 38)

fig.tight_layout()
fig.suptitle("y-Mean by Calo Number" + "   "+ low_E + "-" + high_E + " GeV" + "   "+ low_T + "-" + high_T + "μs", fontsize=40, y=1.05)

ax.legend(loc=0, labelspacing=1.5, framealpha=1, prop={'size':25});
plt.savefig("./mean_image/yMean_by_CaloNum_big.png", bbox_inches='tight')
plt.close()